In [8]:
import requests, json
from zipfile import ZipFile
import pandas as pd
import numpy as np
import os
import tabula as tb
import camelot
import PyPDF2
import decimal

In [ ]:
iteration = 3
with open("json_urls.txt","r") as t_op:
    for line in t_op.readlines():
        try:
            url = requests.get(line.replace("\n",""))
            text = url.text
            data = json.loads(text)
            out_file = f"train{iteration}.json"
            json.dump(data, out_file, indent = 4, ensure_ascii=False)
        except Exception as e:
            print(e)

In [10]:
file = "train6.zip"
with ZipFile(file, 'r') as zip:
    zip.extractall()

In [9]:
for file in os.listdir("train6"):
    data = PyPDF2.PdfFileReader("train6/"+file)
    #df = pd.DataFrame(data)

In [45]:
#print(data)
data.documentInfo
pageObj = data.getPage(0)
 
# extracting text from page
aux=pageObj.extractText().upper()
print(aux)


RE PORT  C ONT AMI N AC I ÓN
Nº:
81596
FAC I LI T Y N AME:
M ILLERHILL RE CY CLING &  E NERGY REC O V ERY  CE NTRE
81597
FAC I L I TY I NS PI R EI D:
UK.SEP A/2 0 0 0 0 2 6 5 1 .FACILITY
C OUNT RY :
UNITED KINGDO M
C ONT I N EN T :
EUROP E
C I T Y:
M ILLERHILL, D ALKEITH
EPR T RSEC TOR C ODE:
5
EPR TR SEC TOR N AME:
WASTE  AND W ASTE WAT ER M ANAGEM ENT
MAI NAC TI V I TY C ODE:
5 (B)
TAR G ETR EAL AS E:
AIR
POL L UTANT:
CARBO N DIOX IDE (CO2 )
EMI S S I ONS :
1 ,3 2 E+ 0 8
DA Y:
4
MONT H:
12
YEA R:
2019
MET EOROLOGI C AL  C ONDI T I ONS
MAX_W I ND_S PEED:
2 ,0 9 E+ 1 5
MI N_W I ND_S PEED:
2 ,4 6 E+ 1 6
AV G _W I ND_S PEED:
2 ,0 7 E+ 1 5
MAX_TEMP:
5 ,7 4 E+ 1 5
MI N_TEMP:
9 ,4 2 E+ 1 5
AV G _TEMP:
7 ,3 1 E+ 1 5
DA YS FOG:
6
RE PORT ER  N AME:
JASO N M O SES
C I T Y_ I D
C6 6 2 B4 B4 D8 5 9 A9 C2 2 4 B5 AC0 ACF2 2 1 7 4 8



In [60]:

def aux2dic(aux):
    sv = False
    res = {}
    for line in aux.split("\n"):
        if ":" in line:
            linea = line.replace(":","").replace(" ","")
            sv = True
            t_n = linea
            break
        elif sv:
            linea = line.replace(":","").replace(" ","")
            sv = False
            dt = linea
            try:
                res[t_n] = int(dt)
            except:
                try:
                    ds = dt.replace(',',".").split("E")
                    dk = float(ds[0])*(10**float(ds[1]))
                    res[t_n] = dk
                except:
                    res[t_n] = dt
            break
    return res

In [61]:
aux2dic(aux)

{}

In [ ]:
def str2tag(y):
    dicY = {}
    print("Creating dictionary")
    for i in tqdm(range(len(y))):
        if y[i] not in dicY:
            dicY[y[i]] = len(dicY)
    print(dicY)
    y2 = []
    print("Changing y")
    for i in tqdm(y):
        y2.append(dicY[i])
    return y2, dicY